# **Chapter 18: Reinforcement Learning**

## **1. Pendahuluan**

Reinforcement Learning (RL) adalah salah satu bidang paling menarik dalam Machine Learning. Berbeda dengan *Supervised Learning* (belajar dari jawaban yang benar) atau *Unsupervised Learning* (mencari pola dalam data tanpa label), RL adalah tentang belajar melalui **trial and error** (coba-coba).

**Konsep Inti:**
* **Agent:** Entitas yang belajar dan membuat keputusan (misal: robot atau karakter game).
* **Environment:** Dunia tempat agen beroperasi (misal: papan permainan atau simulasi fisika).
* **Action:** Tindakan yang dilakukan agen.
* **Reward:** Umpan balik dari lingkungan (positif atau negatif) berdasarkan tindakan agen.
* **Observation:** Informasi yang diterima agen tentang status lingkungan saat ini.

Tujuan agen adalah memaksimalkan total *reward* yang dikumpulkan seiring waktu. Contoh sukses yang terkenal adalah AlphaGo dari DeepMind yang mengalahkan juara dunia Go.

Dalam bab ini, kita akan mempelajari dasar-dasar RL, mulai dari kebijakan sederhana hingga Deep Q-Learning (DQN) yang mampu memainkan game Atari.

## **2. Menyiapkan Lingkungan dengan OpenAI Gym**

Salah satu tantangan dalam RL adalah memiliki lingkungan simulasi yang standar untuk melatih agen. OpenAI Gym (sekarang sering disebut Gymnasium) adalah toolkit standar untuk ini.

Kita akan menggunakan lingkungan klasik **CartPole**. Tujuannya adalah menyeimbangkan tongkat di atas kereta yang bergerak dengan menggerakkan kereta ke kiri atau ke kanan.

In [ ]:
import sys
import gym
import numpy as np
import matplotlib.pyplot as plt

# Membuat lingkungan CartPole
env = gym.make("CartPole-v1", render_mode="rgb_array")

# Inisialisasi lingkungan
obs, info = env.reset(seed=42)

print("Observation shape:", obs.shape)
print("Contoh observasi:", obs)
print("Action space:", env.action_space)

# Visualisasi satu frame
plt.imshow(env.render())
plt.axis("off")
plt.show()

# Contoh mengambil satu tindakan acak
action = 1 # 1 = Gerak Kanan, 0 = Gerak Kiri
obs, reward, done, truncated, info = env.step(action)
print(f"Reward: {reward}, Done: {done}")

**Interpretasi:**
* **Observation:** Array berisi 4 angka: posisi kereta, kecepatan kereta, sudut tongkat, dan kecepatan sudut tongkat.
* **Action:** Diskrit (0 atau 1).
* **Reward:** +1 untuk setiap langkah waktu tongkat tetap tegak.
* **Done:** True jika tongkat jatuh atau kereta keluar dari layar.

## **3. Kebijakan (Policy)**

**Policy** adalah algoritma yang digunakan agen untuk menentukan tindakan berdasarkan observasi. Secara matematis ditulis sebagai $\pi(s) = a$.

### **Hardcoded Policy**
Sebagai permulaan, mari buat kebijakan sederhana: "Jika tongkat miring ke kiri, dorong kereta ke kiri. Jika miring ke kanan, dorong ke kanan." 

In [ ]:
def basic_policy(obs):
    angle = obs[2]
    return 0 if angle < 0 else 1

totals = []
for episode in range(50):
    episode_rewards = 0
    obs, info = env.reset(seed=episode)
    for step in range(200):
        action = basic_policy(obs)
        obs, reward, done, truncated, info = env.step(action)
        episode_rewards += reward
        if done or truncated:
            break
    totals.append(episode_rewards)

print(f"Rata-rata reward (Mean): {np.mean(totals)}")
print(f"Standar Deviasi (Std): {np.std(totals)}")
print(f"Min: {np.min(totals)}, Max: {np.max(totals)}")

**Analisis:**
Kebijakan sederhana ini mungkin menjaga tongkat tetap tegak sebentar, tetapi tidak stabil. Untuk hasil yang lebih baik, kita memerlukan **Neural Network Policy** yang bisa belajar sendiri.

## **4. Neural Network Policy**

Kita bisa menggunakan Neural Network untuk menerima observasi sebagai input dan mengeluarkan probabilitas untuk setiap tindakan.

### **The Credit Assignment Problem**
Jika agen mendapatkan reward +100 setelah 50 langkah, tindakan mana yang bertanggung jawab atas kesuksesan itu? Apakah tindakan terakhir? Atau tindakan 10 langkah lalu?

Ini disebut **Credit Assignment Problem**. Solusi standarnya adalah menggunakan **Discount Factor** ($\gamma$). Reward di masa depan didiskon pada setiap langkah.
* Reward total ($G_t$) = $R_t + \gamma R_{t+1} + \gamma^2 R_{t+2} + \dots$
* Jika $\gamma$ mendekati 0, agen rabun jauh (mementingkan reward instan).
* Jika $\gamma$ mendekati 1, agen memiliki pandangan jauh ke depan.

In [ ]:
import tensorflow as tf
from tensorflow import keras

n_inputs = 4
n_outputs = 2

model = keras.models.Sequential([
    keras.layers.Dense(5, activation="elu", input_shape=[n_inputs]),
    keras.layers.Dense(n_outputs, activation="softmax")
])

def play_one_step(env, obs, model):
    with tf.GradientTape() as tape:
        probas = model(obs[np.newaxis])
        logits = tf.math.log(probas + keras.backend.epsilon())
        action = tf.random.categorical(logits, num_samples=1)
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=action[0], logits=logits
        )
    obs, reward, done, truncated, info = env.step(action[0, 0].numpy())
    return obs, reward, done, truncated, loss, tape

## **5. Policy Gradients (REINFORCE Algorithm)**

Algoritma REINFORCE bekerja dengan cara:
1. Biarkan agen bermain beberapa episode.
2. Hitung total *return* (reward terdiskon) untuk setiap episode.
3. Jika reward tinggi, buat tindakan yang diambil menjadi lebih mungkin terjadi di masa depan.
4. Jika reward rendah, buat tindakan tersebut kurang mungkin terjadi.

Secara teknis, kita menghitung gradien dari loss function, tetapi tidak langsung menerapkannya. Kita mengalikannya dengan *return* sebagai bobot, lalu menghitung rata-ratanya sebelum melakukan update.

In [ ]:
def discount_rewards(rewards, discount_factor):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_factor
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_factor):
    all_discounted_rewards = [
        discount_rewards(rewards, discount_factor)
        for rewards in all_rewards
    ]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [
        (discounted_rewards - reward_mean) / reward_std
        for discounted_rewards in all_discounted_rewards
    ]

## **6. Markov Decision Processes (MDP) & Q-Learning**

$Q(s, a)$ adalah nilai yang diharapkan dari total discounted reward jika agen berada di status $s$, mengambil aksi $a$, dan bertindak optimal setelahnya.

$$Q^*(s, a) = R(s, a) + \gamma \max_{a'} Q^*(s', a')$$

## **7. Deep Q-Learning (DQN)**

DQN menggunakan Neural Network untuk mendekati Q-Function dengan Experience Replay dan Target Network.

In [ ]:
from collections import deque
replay_buffer = deque(maxlen=2000)
print("Komponen DQN siap.")

## **8. Kesimpulan**

Kita telah mempelajari dasar Reinforcement Learning, Policy Gradients, dan Deep Q-Learning.